# **큐레이션 A/B 테스트_ 시뮬레이션 데이터 생성 코드**

In [ ]:
   """
    전처리가 필요한 원본 A/B 테스트 데이터를 생성합니다.
    - 정확한 트래픽 비율(기본 50:50)과 재현성(seed)을 지원
    - 비구매(0)와 결제중 이탈(NaN)을 구분하고, 보조 플래그 컬럼 제공
    - 타임스탬프 기준 시간(now) 외부 주입 가능
    """
#import
import pandas as pd
import numpy as np
import datetime as dt

# 1) 변수 초기화
#session, 트래픽 비율, seed 초기화
def generate_raw_data(
    num_sessions: int = 100000,
    traffic_ratio=(0.5, 0.5),
    seed: int | None = 42,
    now: dt.datetime | None = None,
):

    # 난수생성 및 시간 설정
    rng = np.random.default_rng(seed)
    if now is None:
        now = dt.datetime.now()

    # 그룹별 행동 확률/파라미터
    """
    A - 현재 추석선물에 큐레이션 유지 그룹
    B - ( #감사한 분들께 , #부모님들께) 등과 같은 큐레이션을 넣은 그룹
    """
    params = {
        'A': {'click_proba': 0.15, 'buy_proba': 0.025, 'bounce_proba': 0.35, 'aov': 68000},
        'B': {'click_proba': 0.25, 'buy_proba': 0.031, 'bounce_proba': 0.29, 'aov': 71500}
    }

    # 정확한 트래픽 비율로 그룹 리스트 생성 후 셔플
    nA = int(num_sessions * traffic_ratio[0])
    nB = num_sessions - nA
    groups = np.array(['A'] * nA + ['B'] * nB)
    rng.shuffle(groups)

    # 최근 2주 각 사용자의 접속시간 접속기기 데이터 생성
    max_minutes = 14 * 24 * 60
    minute_offsets = rng.integers(0, max_minutes, size=num_sessions)
    timestamps = np.array([now - dt.timedelta(minutes=int(m)) for m in minute_offsets])

    devices = rng.choice(['mobile', 'desktop'], size=num_sessions, p=[0.8, 0.2])

    # 결과 컨테이너
    session_ids = [f'session_{i}' for i in range(num_sessions)]
    page_views = np.empty(num_sessions, dtype=int)
    clicked_curation = np.zeros(num_sessions, dtype=int)
    purchase_amount = np.zeros(num_sessions, dtype=float)
    purchased_flag = np.zeros(num_sessions, dtype=int)
    abandoned_at_payment = np.zeros(num_sessions, dtype=int)

    # 2) 데이터 생성
    for i in range(num_sessions):
        g = groups[i]
        p = params[g]

        # 이탈 여부
        is_bounce = rng.random() < p['bounce_proba']
        if is_bounce:
            page_views[i] = 1
            clicked_curation[i] = 0
            # purchase_amount[i] already 0.0
            continue

        # 비이탈 세션: 페이지뷰 랜덤(2~9)
        page_views[i] = rng.integers(2, 10)

        # 클릭
        clicked_curation[i] = 1 if rng.random() < p['click_proba'] else 0

        # 구매
        if rng.random() < p['buy_proba']:
            # 결제 중 이탈(결측)
            if rng.random() < 0.1:
                purchase_amount[i] = np.nan
                abandoned_at_payment[i] = 1
            else:
                amt = rng.normal(loc=p['aov'], scale=15000)
                amt = max(10_000, amt)
                purchase_amount[i] = round(amt)  # 원 단위 정수
                purchased_flag[i] = 1
        # else: remain 0.0

    df = pd.DataFrame({
        'session_id': session_ids,
        'timestamp': timestamps,
        'user_group': groups,
        'device': devices,
        'page_views': page_views,
        'clicked_curation': clicked_curation,
        'purchase_amount': purchase_amount,  # 0=비구매, NaN=결제중 이탈, >0=구매성공
        'purchased_flag': purchased_flag,    # 명시적 구매 성공 여부
        'abandoned_at_payment': abandoned_at_payment,  # 결제 단계 이탈(결측 발생)
    })

    return df

# 3) 파일 저장 실행
generate_raw_data()
df = generate_raw_data()
df.to_csv('raw_ab_test_data.csv', index=False)
df